# Electrode Config

This notebook is used for managing electrode configurations within a study

In order to fetch data from the Cionic REST APIs, you need to be authenticated.  
* Login to [cionic.com](https://cionic.com/a)  
* download a [token file](https://cionic.com/a/v0.7/accounts/@me/token.json)  
* save it to the root of your CDE directory  

```
note this credential file should never be checked in or shared
```

The default behavior is to run as **prepare_only** 
which will generate the notebooks but not execute them.

Clink on the linked title and run the generated notebook.

## Parameters

```
orgid        | the organization that hosts the study
study        | which study you want to run over    
collection   | optional collection number to run for a specific collection (0 means all)
notebook     | which notebook you want to run                   
execute      | parameterizes the notebook and executes it 
overwrite    | delete previously saved parameterized notebook 
limit        | how many collection records to run
```


In [ ]:
import json
import os

from IPython.display import clear_output, display
import ipywidgets as widgets
from ipywidgets import Layout, interact

import cionic

tokenpath = os.path.abspath('../token.json')

d = cionic.auth(tokenpath=tokenpath)
orgs = [org['shortname'] for org in d]

org_select = widgets.Dropdown(options=orgs, description="Org ID")
study_select = widgets.Dropdown(options=[], description="Study")
config_select = widgets.Dropdown(options=[], description="Configs")
config_entry = widgets.Textarea(
    rows=30, description="Entry", layout=Layout(width='1200px')
)
config_name = widgets.Text(description="Config Name")


global saved_orgid, saved_study, saved_config, studies, configs, saved_sxid, saved_dcxid
saved_orgid = None
saved_study = None
saved_config = None
studies = []
configs = []
saved_sxid = None
saved_dcxid = None


@interact
def show_runner(
    orgid=org_select,
    study=study_select,
    config=config_select,
    name=config_name,
    entry=config_entry,
):
    print(orgid, study, config)
    global saved_orgid, saved_study, saved_config, studies, configs, saved_sxid, saved_dcxid

    def update_button_clicked(b):
        with output:
            clear_output()
            payload = {
                'config_name': config_name.value,
                'config': json.loads(config_entry.value),
            }
            cionic.put_cionic(
                f"{orgid}/studies/{saved_sxid}/deviceconfig/{saved_dcxid}",
                json=payload,
                ver='1.16',
            )

    def new_button_clicked(b):
        with output:
            clear_output()
            payload = {
                'config_name': config_name.value,
                'config': json.loads(config_entry.value),
            }
            cionic.post_cionic(
                f"{orgid}/studies/{saved_sxid}/deviceconfig/", json=payload, ver='1.16'
            )

    update_button = widgets.Button(description='Update')
    update_button.on_click(update_button_clicked)
    new_button = widgets.Button(description='Add New')
    new_button.on_click(new_button_clicked)
    output = widgets.Output()
    display(update_button, new_button, output)

    if saved_orgid != orgid:
        saved_orgid = orgid
        studies = cionic.get_cionic(f'{orgid}/studies')
        titles = [s['shortname'] for s in studies]
        study_select.options = titles
        saved_orgid = orgid
        return

    if saved_study != study:
        saved_study = study
        saved_sxid = None  # set below

        for s in studies:
            if s['shortname'] == study:
                saved_sxid = s['xid']
                configs = cionic.get_cionic(
                    f"{orgid}/studies/{saved_sxid}/deviceconfig", ver='1.16'
                )
                names = ['__new__'] + [config['config_name'] for config in configs]
                config_select.options = names
        return

    if saved_config != config:
        saved_config = config
        saved_dcxid = None
        if config == '__new__':
            config_name.value = ""
            config_entry.value = json.dumps({}, indent=2)
        else:
            config_name.value = config
            for c in configs:
                if c['config_name'] == config:
                    saved_dcxid = c['xid']
                    config_entry.value = json.dumps(c['config'], indent=2)